In [49]:
!pip install gurobipy

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

In [51]:
df1=pd.read_csv('TX_suppliers.csv')
df2=pd.read_csv('TX_hubs.csv')
df3=pd.read_csv('TX_plants.csv')
df4=pd.read_csv('TX_roads.csv')
df5=pd.read_csv('TX_railroads.csv')
df5

,index,hub,plant,distance,cost,loading,capacity(Mg)
0,1,17201,541,922.023251,17.095711,3066792,338000
1,2,17201,542,1160.842967,20.923752,3066792,338000
2,3,17201,543,186.924258,5.312809,3066792,338000
3,4,17201,544,1074.699073,19.542951,3066792,338000
4,5,17201,545,1160.842967,20.923752,3066792,338000
...,...,...,...,...,...,...,...
5506,5507,18303,10061,1132.963314,20.476869,3066792,338000
5507,5508,18303,10062,1330.810658,23.648164,3066792,338000
5508,5509,18303,10063,1223.144021,21.922376,3066792,338000
5509,5510,18303,10065,1717.592064,29.847883,3066792,338000


In [52]:
suppliers1=df1.dropna(subset=['county'])
suppliers=suppliers1['county']

In [53]:
hubs=df2['hub']
hub_invest=df2['invest']
hub_capacity=df2['capacity(Mg)']

In [54]:
plants=df3['plant']
plant_invest=df3['invest']
plant_capacity=df3['capacity']

In [55]:
truck_transportation_cost=df4['cost']
train_transportation_cost=df5['cost']
train_load_cost=df5['loading']
train_capacity=df5['capacity(Mg)']

In [56]:
cost_in_truck={(row['county'].astype(int),row['hub'].astype(int)): row['cost'] for _, row in df4.iterrows()}
cost_in_train={(row['hub'].astype(int),row['plant'].astype(int)): row['cost'] for _, row in df5.iterrows()}
a = list(cost_in_truck.items())[:5]
print("(supplier Id,","Hub Id)" " cost")
for key, value in a:
    print(key , value)
print("(Hub Id,","Plant Id)" " cost")
b = list(cost_in_train.items())[:5]
for key, value in b:
    print(key, value)

(supplier Id, Hub Id) cost
(48001, 17201) 40.5806912
(48001, 17218) 47.66127744
(48001, 17359) 64.26784
(48001, 17372) 72.00328576
(48001, 17395) 37.3949072
(Hub Id, Plant Id) cost
(17201, 541) 17.09571069
(17201, 542) 20.92375192
(17201, 543) 5.312808937
(17201, 544) 19.54295144
(17201, 545) 20.92375192


In [57]:
supplier_dict={row['county']:row['supply'] for _,row in suppliers1.iterrows()}
hubs_dict={row['hub']:row['capacity(Mg)'] for _,row in df2.iterrows()}
plants_dict={row['plant']:row['capacity'] for _,row in df3.iterrows()}
print("supplier Id," " Supply")
c = list(supplier_dict.items())[:5]
for key, value in c:
    print(key, value)
d = list(hubs_dict.items())[:5]
print("Hub Id," " capacity")
for key, value in d:
    print(key, value)
e = list(plants_dict.items())[:5]
print("Plant Id," " capacity")
for key, value in e:
    print(key, value)

supplier Id, Supply
48001.0 13131.97171
48003.0 1177.35195
48005.0 3854.618542
48007.0 308.1826286
48009.0 19802.13651
Hub Id, capacity
17201.0 300000.0
17218.0 300000.0
17359.0 300000.0
17372.0 300000.0
17395.0 300000.0
Plant Id, capacity
541.0 655447.0
542.0 655447.0
543.0 655447.0
544.0 655447.0
545.0 655447.0


In [58]:
suppliers_list=[key for key in supplier_dict.keys()]
hubs_list=[key for key in hubs_dict.keys()]
plants_list=[key for key in plants_dict.keys()]
third_party_cost=[500,1000,2000]
suppliers_list

[48001.0,
 48003.0,
 48005.0,
 48007.0,
 48009.0,
 48011.0,
 48013.0,
 48015.0,
 48017.0,
 48019.0,
 48021.0,
 48023.0,
 48025.0,
 48027.0,
 48029.0,
 48031.0,
 48033.0,
 48035.0,
 48037.0,
 48039.0,
 48041.0,
 48043.0,
 48045.0,
 48047.0,
 48049.0,
 48051.0,
 48053.0,
 48055.0,
 48057.0,
 48059.0,
 48061.0,
 48063.0,
 48065.0,
 48067.0,
 48069.0,
 48071.0,
 48073.0,
 48075.0,
 48077.0,
 48079.0,
 48081.0,
 48083.0,
 48085.0,
 48087.0,
 48089.0,
 48091.0,
 48093.0,
 48095.0,
 48097.0,
 48099.0,
 48101.0,
 48103.0,
 48105.0,
 48107.0,
 48109.0,
 48111.0,
 48113.0,
 48115.0,
 48117.0,
 48119.0,
 48121.0,
 48123.0,
 48125.0,
 48127.0,
 48129.0,
 48131.0,
 48133.0,
 48135.0,
 48137.0,
 48139.0,
 48141.0,
 48143.0,
 48145.0,
 48147.0,
 48149.0,
 48151.0,
 48153.0,
 48155.0,
 48157.0,
 48159.0,
 48161.0,
 48163.0,
 48165.0,
 48167.0,
 48169.0,
 48171.0,
 48173.0,
 48175.0,
 48177.0,
 48179.0,
 48181.0,
 48183.0,
 48185.0,
 48187.0,
 48189.0,
 48191.0,
 48193.0,
 48195.0,
 48197.0,
 48199.0,


In [59]:
model = gp.Model("Bioethanol_Supply_Chain")

In [60]:
# Decision variables
yj= {}  # Binary variable indicating if a hub is opened at location j
zk= {}  # Binary variable indicating if a plant is opened at location k

In [61]:
# Create decision variables and add them to the model
for j in hubs_list:
    yj[j] = model.addVar(vtype=gp.GRB.BINARY, name=f"y_{j}")
for k in plants_list:
    zk[k] = model.addVar(vtype=gp.GRB.BINARY, name=f"z_{k}")
Sij = model.addVars( suppliers_list, hubs_list )
Sjk = model.addVars( hubs_list, plants_list)
St = model.addVar( vtype=gp.GRB.INTEGER)

In [62]:
# Set objective
for T in third_party_cost:
  model.setObjective(
    gp.quicksum(3476219* yj[j] for j in hubs_list)
    + gp.quicksum(130956797 * zk[k] for k in plants_list)
    + gp.quicksum(cost_in_truck[i,j] * Sij[i,j] for i,j in cost_in_truck.keys())
    + gp.quicksum((cost_in_train[j,k] * Sjk[j,k])+3066792 for j,k in cost_in_train.keys())
    +(St*T),
    gp.GRB.MINIMIZE,
)

In [63]:
model.addConstrs( gp.quicksum( Sij[i,j] for j in hubs_list ) <= supplier_dict[i] for i in suppliers_list )
model.addConstrs( gp.quicksum( Sij[i,j] for i in suppliers_list ) <= hubs_dict[j]*yj[j] for j in hubs_list )
model.addConstrs( gp.quicksum( Sjk[j,k] for j in hubs_list ) <= plants_dict[k]*zk[k] for k in plants_list )
model.addConstrs( gp.quicksum( Sjk[j,k]*zk[k] for j in hubs_list ) == (1476310602/232) for k in plants_list )
model.addConstrs(Sjk[j,k]<=338000 for j in hubs_list for k in plants_list)
model.addConstrs(gp.quicksum( St+Sij[i,j] for i in suppliers_list) == gp.quicksum( Sjk[j,k] for k in plants_list) for j in hubs_list)
model.addConstrs(gp.quicksum(Sij[i,j]for i in suppliers_list) >= 0 for j in hubs_list)
model.addConstrs(gp.quicksum(Sjk[j,k]for j in hubs_list) >= 0 for k in plants_list)
model.addConstrs(gp.quicksum(Sjk[j,k]for j in hubs_list) >= 0 for k in plants_list)
model.addConstrs(St >= (6363408-(gp.quicksum(Sij[i,j] for i in suppliers_list )))for j in hubs_list)

{17201.0: <gurobi.Constr *Awaiting Model Update*>,
 17218.0: <gurobi.Constr *Awaiting Model Update*>,
 17359.0: <gurobi.Constr *Awaiting Model Update*>,
 17372.0: <gurobi.Constr *Awaiting Model Update*>,
 17395.0: <gurobi.Constr *Awaiting Model Update*>,
 17404.0: <gurobi.Constr *Awaiting Model Update*>,
 17447.0: <gurobi.Constr *Awaiting Model Update*>,
 17466.0: <gurobi.Constr *Awaiting Model Update*>,
 17507.0: <gurobi.Constr *Awaiting Model Update*>,
 17592.0: <gurobi.Constr *Awaiting Model Update*>,
 17620.0: <gurobi.Constr *Awaiting Model Update*>,
 17679.0: <gurobi.Constr *Awaiting Model Update*>,
 17717.0: <gurobi.Constr *Awaiting Model Update*>,
 17784.0: <gurobi.Constr *Awaiting Model Update*>,
 17792.0: <gurobi.Constr *Awaiting Model Update*>,
 17822.0: <gurobi.Constr *Awaiting Model Update*>,
 17829.0: <gurobi.Constr *Awaiting Model Update*>,
 17896.0: <gurobi.Constr *Awaiting Model Update*>,
 17931.0: <gurobi.Constr *Awaiting Model Update*>,
 17934.0: <gurobi.Constr *Await

In [64]:
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads



GurobiError: ignored